In [1]:
# ================================================
# 🧩 STEP 1: Install Dependencies
# ================================================
!pip install -q transformers peft accelerate datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.5 MB/s eta 0:00:00


# New Section

In [2]:
# ================================================
# 🧩 STEP 2: Import Libraries
# ================================================
import torch
#from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [3]:
from datasets import Dataset
#dataset = Dataset.from_list(data)

from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/boot_recommendations.json")

print(dataset)

dataset = dataset["train"].train_test_split(test_size=0.2)

print(dataset)
# Split for training/evaluation
#dataset = dataset.train_test_split(test_size=0.2)

# ================================================
# 🧩 STEP 4: Load Base Model and Tokenizer
# ================================================
model_name = "facebook/opt-1.3b"  # You can use "facebook/opt-350m" for Colab CPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 2000
    })
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['instruction', 'response'],
        num_rows: 400
    })
})


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [4]:
# ================================================
# 🧩 STEP 5: Tokenize Data
# ================================================
def format_example(example):
    return f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}"

def tokenize_function(example):
    text = format_example(example)
    tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [5]:
# ================================================
# 🧩 STEP 6: Prepare LoRA Configuration
# ================================================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # For OPT models
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


In [6]:
# ================================================
# 🧩 STEP 7: Load Model and Apply LoRA
# ================================================
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Optional, for memory efficiency
    device_map="auto"
)
model = get_peft_model(model, lora_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
!pip install --upgrade transformers

# ================================================
# 🧩 STEP 8: Training Arguments
# ================================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    eval_strategy="steps",
    fp16=True,
)

In [8]:
# ================================================
# 🧩 STEP 9: Trainer Setup
# ================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: silam (silam-red-wing-shoes) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
10,9.791300,5.226368
20,1.643100,0.403613
30,0.366100,0.303100
40,0.268600,0.218695
50,0.198300,0.149711
60,0.142900,0.099347
70,0.098800,0.067771
80,0.069900,0.054411
90,0.057600,0.048837
100,0.052400,0.044139


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

Step,Training Loss,Validation Loss
10,9.791300,5.226368
20,1.643100,0.403613
30,0.366100,0.303100
40,0.268600,0.218695
50,0.198300,0.149711
60,0.142900,0.099347
70,0.098800,0.067771
80,0.069900,0.054411
90,0.057600,0.048837
100,0.052400,0.044139


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
# ================================================
# 🧩 STEP 10: Save Fine-Tuned Model
# ================================================
trainer.save_model("./lora-boot-recommendation-finetuned-model")
tokenizer.save_pretrained("./lora-boot-recommendation-finetuned-pretrained-model")


('./lora-finetuned-model/tokenizer_config.json',
 './lora-finetuned-model/special_tokens_map.json',
 './lora-finetuned-model/vocab.json',
 './lora-finetuned-model/merges.txt',
 './lora-finetuned-model/added_tokens.json',
 './lora-finetuned-model/tokenizer.json')

In [ ]:
# ================================================
# 🧩 STEP 11: Test the Model
# ================================================
from transformers import pipeline

pipe = pipeline("text-generation", model="./lora-boot-finetuned-model", tokenizer=tokenizer, max_new_tokens=100)

prompt = "### Instruction:\nHow do I update my shipping address?\n### Response:\n"
result = pipe(prompt)
print(result[0]['generated_text'])

Device set to use cpu
Both `max_new_tokens` (=100) and `max_length`(=21) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


### Instruction:
How do I update my shipping address?
### Response:

**Update: You can update your address in your My Account section.

For your security, please do not change the shipping address before you check out.

**Update: You can view your shipping address in your My Account section.


In [ ]:
# ================================================
# 🧩 STEP 11: Test the Model
# ================================================
from transformers import pipeline

pipe = pipeline("text-generation", model="./lora-finetuned-model", tokenizer=tokenizer, max_new_tokens=100)

prompt = "### Instruction:\nHow do I update my shipping address?\n### Response:\n"
result = pipe(prompt)
print(result[0]['generated_text'])

Device set to use cpu
Both `max_new_tokens` (=100) and `max_length`(=21) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


### Instruction:
How do I update my shipping address?
### Response:
You can update your address when you order the item.
### Payment:
I will pay with a bank transfer and receive the item within 30 days of receiving the payment.

### Shipping:
I will ship the item via UPS.

Shipping costs:
I will send the items by airmail, to the address provided.

Shipping costs will be charged when the order is shipped and you will receive an invoice.

Shipping costs are calculated based on the country of destination


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
hf_token = userdata.get("HF_TOKEN")
from huggingface_hub import login
login(hf_token)

model_name = "./lora-finetuned-model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.push_to_hub("sichilam/boot-recommendation-llm")
tokenizer.push_to_hub("sichilam/boot-recommendation-llm")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   9%|9         |  576kB / 6.30MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sichilam/boot-recommendation-llm/commit/22371866079229197aa702aeb159da3c9d09ac17', commit_message='Upload tokenizer', commit_description='', oid='22371866079229197aa702aeb159da3c9d09ac17', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sichilam/boot-recommendation-llm', endpoint='https://huggingface.co', repo_type='model', repo_id='sichilam/boot-recommendation-llm'), pr_revision=None, pr_num=None)